# Exercice 4: Attaquer le système en modifiant directement le fichier de données, en changeant le montant d’une transaction.


In [23]:
import os
current_dir = os.getcwd()
new_dir = current_dir.replace("\\tests", "")
os.chdir(new_dir)
from services.transaction_service import TransactionService

In [26]:
# Old transaction value
transaction_service = TransactionService()
transaction = transaction_service.get_transaction(1)
print(transaction)

 2023-11-13 21:36:58.939091 - (id: 1) John Doe 1350.0€ -> (id: 2) Jane Dupont 930.0€: 500.0€


In [29]:
# Update transaction amount
transaction.amount = 1000
transaction_service.update_transaction(transaction)

In [30]:
# New transaction value
transaction = transaction_service.get_transaction(1)
print(transaction)

 2023-11-13 21:36:58.939091 - (id: 1) John Doe 1350.0€ -> (id: 2) Jane Dupont 930.0€: 1000.0€


# Exercice 7. Vérifiez que l’attaque précédente ne fonctionne plus.

In [31]:
import os
current_dir = os.getcwd()
new_dir = current_dir.replace("\\tests", "")
os.chdir(new_dir)
from services.transaction_service import TransactionService

In [32]:
# Old transaction value
transaction_service = TransactionService()
transaction = transaction_service.get_transaction(1)
print(transaction)

 2023-11-13 21:36:58.939091 - (id: 1) John Doe 1350.0€ -> (id: 2) Jane Dupont 930.0€: 1000.0€


In [33]:
# Update transaction amount
transaction.amount = 1000
transaction_service.update_transaction(transaction)

In [34]:
# New transaction value
transaction = transaction_service.get_transaction(1)
print(transaction)

 2023-11-13 21:36:58.939091 - (id: 1) John Doe 1350.0€ -> (id: 2) Jane Dupont 930.0€: 1000.0€
